Cleaning this data of Arabic news labeled for category: https://data.mendeley.com/datasets/322pzsdxwy/1

Their description of the data:

> RTAnews dataset is a collections of multi-label Arabic texts, collected form Russia Today in Arabic news portal. It consists of 23,837 texts (news articles) distributed over 40 categories, and divided into 15,001 texts for the training and 8,836 texts for the test.
The original dataset (without preprocessing), a preprocessed version of the dataset, versions of the dataset in MEKA and Mulan formats, single-label version, and WEAK version all are available.

I'm using the preprocessed data here, which has all punctuation removed and joins the title with the main text. Later, could be interested to go back and look at title versus text.

Note that these aren't full news articles, more like paragraph-lengh blurbs. Maybe the first para of the scraped articles?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob2
#from tqdm import tqdm

%matplotlib inline
from IPython.core.pylabtools import figsize

In [2]:
path = '/Users/awhite/Downloads/RTAnews-4/training/'
folders = [x[0] for x in os.walk(path)]
del folders[0] 
folders

['/Users/awhite/Downloads/RTAnews-4/training/اتفاق_ايران_النووي',
 '/Users/awhite/Downloads/RTAnews-4/training/استفتاء_بريطانيا',
 '/Users/awhite/Downloads/RTAnews-4/training/اكتشافات',
 '/Users/awhite/Downloads/RTAnews-4/training/الاعتراف_بدولة_فلسطين',
 '/Users/awhite/Downloads/RTAnews-4/training/الانتخابات_الأمريكية',
 '/Users/awhite/Downloads/RTAnews-4/training/الأزمة_الأوكرانية',
 '/Users/awhite/Downloads/RTAnews-4/training/الأزمة_السورية',
 '/Users/awhite/Downloads/RTAnews-4/training/الأزمة_اليمنية',
 '/Users/awhite/Downloads/RTAnews-4/training/البحوث_الطبية',
 '/Users/awhite/Downloads/RTAnews-4/training/البورصات',
 '/Users/awhite/Downloads/RTAnews-4/training/التقنية_والمعلومات',
 '/Users/awhite/Downloads/RTAnews-4/training/السياحة_في_العالم',
 '/Users/awhite/Downloads/RTAnews-4/training/المعارضة_السورية',
 '/Users/awhite/Downloads/RTAnews-4/training/الهجرة_إلى_أوروبا',
 '/Users/awhite/Downloads/RTAnews-4/training/امراض',
 '/Users/awhite/Downloads/RTAnews-4/training/انقلاب_تركيا'

In [3]:
#Quick test to be sure this is working
folders_test = folders[0:2]
folders_test

['/Users/awhite/Downloads/RTAnews-4/training/اتفاق_ايران_النووي',
 '/Users/awhite/Downloads/RTAnews-4/training/استفتاء_بريطانيا']

In [4]:
media_df = pd.DataFrame()

for folder in folders_test:
    corpus = []
    file_list = glob2.glob(folder + '/*.txt')
    for file_path in file_list:
        with open(file_path) as f_input:
            corpus.append(f_input.read())
            df = pd.DataFrame(corpus)
            df['category'] = folder.replace(path,'')
    media_df = media_df.append(df)

In [5]:
len(media_df)

265

In [6]:
media_df.category.value_counts()

اتفاق_ايران_النووي    155
استفتاء_بريطانيا      110
Name: category, dtype: int64

In [7]:
#Let's load everything!
train = pd.DataFrame()

for folder in folders:
    corpus = []
    file_list = glob2.glob(folder + '/*.txt')
    for file_path in file_list:
        with open(file_path) as f_input:
            corpus.append(f_input.read())
            df = pd.DataFrame(corpus)
            df['category'] = folder.replace(path,'')
    train = train.append(df)
    

test_path = '/Users/awhite/Downloads/RTAnews-4/test/'
test_folders = [x[0] for x in os.walk(test_path)]
del test_folders[0] 

test = pd.DataFrame()

for folder in test_folders:
    corpus = []
    file_list = glob2.glob(folder + '/*.txt')
    for file_path in file_list:
        with open(file_path) as f_input:
            corpus.append(f_input.read())
            df = pd.DataFrame(corpus)
            df['category'] = folder.replace(test_path,'')
    test = test.append(df)

In [8]:
len(train)

16610

In [9]:
train.head()

,0,category
0,كارتر يتوجه رياض اطار جولته اوسط اعلنت مستشاره...,اتفاق_ايران_النووي
1,خامنئي يشيد بجهود مفاوض فيينا اشاد مرشد اعلى ث...,اتفاق_ايران_النووي
2,يتراجع مخاوف تدفق سريع ايراني هبطت اسعار ارتفا...,اتفاق_ايران_النووي
3,اقتصاد الماني يزور ايران غضون ايام ينوي اقتصاد...,اتفاق_ايران_النووي
4,خامنئي اتفاق مبرم ايران عالميه اخضاعه متمعن زع...,اتفاق_ايران_النووي


In [10]:
len(test)

11060

In [11]:
test.head()

,0,category
0,نووي ايراني ماراثون تفاوض قطعت مفاوضات نوويه ا...,اتفاق_ايران_النووي
1,لافروف يبحث اوزبكستان وضع آسيا وسطى تعاون ثنائ...,اتفاق_ايران_النووي
2,تبادل تجاري روسيا ايران يصل مليارات دولار اعلن...,اتفاق_ايران_النووي
3,سعوديه تويد اتفاق نووي بشروط واشنطن توفد زير د...,اتفاق_ايران_النووي
4,تصبح ايران نقطه تقا موسكو واشنطن لتسويه ازمات ...,اتفاق_ايران_النووي


In [12]:
#Our counts don't exactly match their claimed train/test split.
#For now, let's join the data and split it ourselves.
media_df = train.append(test)
media_df = media_df.rename(columns = {0:'text'})

In [13]:
len(media_df)

27670

In [14]:
media_df.category.value_counts()

كرة_القدم                       4301
الأزمة_السورية                 3160
جماعات_مسلحة                    2142
أسواق_النفط                    1364
لاجئون                         1256
رياضات_اخرى                     1100
المعارضة_السورية                1002
صواريخ                           741
التقنية_والمعلومات               639
فضاء                             630
أسلحة_ومعدات_عسكرية             625
الهجرة_إلى_أوروبا              616
الأزمة_الأوكرانية              595
اكتشافات                         577
الأزمة_اليمنية                  566
تفجيرات                          559
مشاهير                           539
البحوث_الطبية                    530
معلومات_عامة                     484
طائرات_حربية                    479
الانتخابات_الأمريكية            447
جرائم                           433
مظاهرات                          422
مؤشرات_اقتصادية                 400
عقوبات_اقتصادية                  397
الاعتراف_بدولة_فلسطين            369
امراض                            323
أولمبياد_ريو_د

In [15]:
media_df.to_csv('/Users/awhite/Documents/snorkel/arabic_news_cleaned.csv', index=False)